In [282]:
import spotipy
from spotipy import util
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import pandas as pd
import matplotlib.pyplot as plt
import urllib

## REMOVE BEFORE PUSHING ##
CLIENT_ID='472796d4b7904eb8ab972808d46bd0b0'
CLIENT_SECRET='cc89036aabd04a61a9d20970a0510186'



username = 'elw86ve5g5t944wwlef6qyzu3' # Alex Fioto's user id
modify_playlist = 'playlist-modify-public'
user_auth = 'user-top-read'
redirect_uri = 'http://127.0.0.1:8080'
toby = '1220960127'

token = util.prompt_for_user_token(username=username,
                                   scope=modify_playlist + ' ' + user_auth,
                                   client_id=CLIENT_ID,
                                   client_secret=CLIENT_SECRET,
                                   redirect_uri=redirect_uri) 

sp = spotipy.Spotify(auth=token)


In [283]:
logslaught = 'spotify:artist:1I471vwcRhqQl6QonGZlen'
grandson = 'spotify:artist:4ZgQDCtRqZlhLswVS6MHN4'
run_the_jewels = 'spotify:artist:4RnBFZRiMLRyZy0AzzTg2C'
hatebreed = 'spotify:artist:17Mb968quDHpjCkIyq30QV'

In [285]:
def get_artist_tracks(sp, artists, n_albums=100,):
    '''
    Function takes in the Spotify URI of one or more artists and returns a Pandas dataframe with Spotify's proprietary audio features.
    
    n_albums: number of albums to fetch per artist
    '''
    
    uris = []
    album_uris = []
    df_list = []
    
    # Checking if user input artists as a list. 
    if type(artists) != list:
        # Rectifying if user did not input list
        artists = [artists]
    
    for artist in artists:
        albums = sp.artist_albums(artist)
        for album in albums['items']:
            album_uris.append(album['uri'])
    
    for uri in album_uris:
        album = sp.album(uri)
        album_name = album['name']
        
        album_artist = ', '.join([artist['name'] for artist in album['artists']])
        
        tracks = sp.album_tracks(uri)
        audio_features_dict = sp.audio_features()
        df = pd.DataFrame.from_dict(tracks['items'])
        audio_features_df = pd.DataFrame.from_dict(sp.audio_features(list(df['uri'])))
        audio_features_df['track_name'] = df['name']
        audio_features_df['album_name'] = album_name
        audio_features_df['artist_name'] = album_artist
        df_list.append(audio_features_df)
    
    df = pd.concat(df_list)
    df.reset_index(inplace=True, drop=True)
    df.drop_duplicates()
    return df

In [286]:
df = get_artist_tracks(sp, artists = [grandson, run_the_jewels, hatebreed])

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,track_name,album_name,artist_name
0,0.340,0.539,5,-7.286,1,0.0419,0.6310000,0.0000225,0.8440,0.159,...,audio_features,4tlkasH25ihwA68F1Xw8wI,spotify:track:4tlkasH25ihwA68F1Xw8wI,https://api.spotify.com/v1/tracks/4tlkasH25ihw...,https://api.spotify.com/v1/audio-analysis/4tlk...,150727,4,Death Of An Optimist // Intro,Death Of An Optimist,grandson
1,0.582,0.817,0,-6.475,1,0.0399,0.0017000,0.0158000,0.5750,0.622,...,audio_features,3r0mb1tzAcCNoA8YRXuIhZ,spotify:track:3r0mb1tzAcCNoA8YRXuIhZ,https://api.spotify.com/v1/tracks/3r0mb1tzAcCN...,https://api.spotify.com/v1/audio-analysis/3r0m...,198366,4,In Over My Head,Death Of An Optimist,grandson
2,0.495,0.731,8,-6.304,1,0.0458,0.0003480,0.0000000,0.0598,0.284,...,audio_features,73RdXBA5oTmOr6furSPoti,spotify:track:73RdXBA5oTmOr6furSPoti,https://api.spotify.com/v1/tracks/73RdXBA5oTmO...,https://api.spotify.com/v1/audio-analysis/73Rd...,216000,4,Identity,Death Of An Optimist,grandson
3,0.353,0.802,7,-6.996,1,0.1390,0.0071100,0.0000000,0.3520,0.447,...,audio_features,6Ndu96RYi1fvFBMNCYLIbT,spotify:track:6Ndu96RYi1fvFBMNCYLIbT,https://api.spotify.com/v1/tracks/6Ndu96RYi1fv...,https://api.spotify.com/v1/audio-analysis/6Ndu...,205426,4,Left Behind,Death Of An Optimist,grandson
4,0.536,0.845,2,-5.765,1,0.0615,0.0033000,0.0000000,0.0465,0.600,...,audio_features,3pShTDa5E1bPBkrc1mxxGY,spotify:track:3pShTDa5E1bPBkrc1mxxGY,https://api.spotify.com/v1/tracks/3pShTDa5E1bP...,https://api.spotify.com/v1/audio-analysis/3pSh...,208296,4,Dirty,Death Of An Optimist,grandson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,0.468,0.574,0,-14.319,1,0.0359,0.0000107,0.0840000,0.1140,0.384,...,audio_features,6tjO14edAkFiYhIHRz64gR,spotify:track:6tjO14edAkFiYhIHRz64gR,https://api.spotify.com/v1/tracks/6tjO14edAkFi...,https://api.spotify.com/v1/audio-analysis/6tjO...,93467,3,Under The Knife,Under The Knife,Hatebreed
556,0.397,0.629,1,-13.305,1,0.0677,0.0000449,0.1900000,0.0579,0.476,...,audio_features,6PnIH43bFQSHoZqB4sJJIV,spotify:track:6PnIH43bFQSHoZqB4sJJIV,https://api.spotify.com/v1/tracks/6PnIH43bFQSH...,https://api.spotify.com/v1/audio-analysis/6PnI...,102853,4,Filth,Under The Knife,Hatebreed
557,0.423,0.720,1,-9.520,1,0.1650,0.0037000,0.2750000,0.0947,0.595,...,audio_features,7msqBY84RdipiTi6cC6K0U,spotify:track:7msqBY84RdipiTi6cC6K0U,https://api.spotify.com/v1/tracks/7msqBY84Rdip...,https://api.spotify.com/v1/audio-analysis/7msq...,129680,4,Not One Truth,Under The Knife,Hatebreed
558,0.460,0.709,5,-11.808,0,0.0535,0.0030300,0.2190000,0.1890,0.715,...,audio_features,2jJj23XAgILAPtr6q9CaGl,spotify:track:2jJj23XAgILAPtr6q9CaGl,https://api.spotify.com/v1/tracks/2jJj23XAgILA...,https://api.spotify.com/v1/audio-analysis/2jJj...,160760,4,Severed,Under The Knife,Hatebreed


In [231]:
print(len(df))
df.head(2)

46


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,track_name,album_name,artist_name
0,0.340,0.539,5,-7.286,1,0.0419,0.6310,0.0000225,0.844,0.159,...,audio_features,4tlkasH25ihwA68F1Xw8wI,spotify:track:4tlkasH25ihwA68F1Xw8wI,https://api.spotify.com/v1/tracks/4tlkasH25ihw...,https://api.spotify.com/v1/audio-analysis/4tlk...,150727,4,Death Of An Optimist // Intro,Death Of An Optimist,grandson
1,0.582,0.817,0,-6.475,1,0.0399,0.0017,0.0158000,0.575,0.622,...,audio_features,3r0mb1tzAcCNoA8YRXuIhZ,spotify:track:3r0mb1tzAcCNoA8YRXuIhZ,https://api.spotify.com/v1/tracks/3r0mb1tzAcCN...,https://api.spotify.com/v1/audio-analysis/3r0m...,198366,4,In Over My Head,Death Of An Optimist,grandson


In [258]:
def get_user_tracks(sp, limit=10):
    user_tracks = sp.current_user_top_tracks(limit=limit)
    audio_features =  sp.audio_features([item['uri'] for item in user_tracks['items']])
    names = [item['name'] for item in user_tracks['items']]
    df = pd.DataFrame.from_dict(audio_features)
    df['track_name'] = names
    return df

In [259]:
user_df = get_user_tracks(sp = sp)

In [260]:
user_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,track_name
0,0.817,0.814,2,-3.985,1,0.1850,0.3200,0.00000000,0.1520,0.247,109.99400000000000,audio_features,5rLyYxZNzca00ENADO9m54,spotify:track:5rLyYxZNzca00ENADO9m54,https://api.spotify.com/v1/tracks/5rLyYxZNzca0...,https://api.spotify.com/v1/audio-analysis/5rLy...,196800,4,When I Grow Up
1,0.701,0.525,1,-7.938,0,0.3820,0.0353,0.00000255,0.0615,0.385,75.69300000000000,audio_features,5jQYkYhoOlBW4vJ2l4TCxl,spotify:track:5jQYkYhoOlBW4vJ2l4TCxl,https://api.spotify.com/v1/tracks/5jQYkYhoOlBW...,https://api.spotify.com/v1/audio-analysis/5jQY...,152253,4,"Blockbuster Night, Pt. 1"
2,0.727,0.642,1,-4.915,1,0.0870,0.0291,0.00000000,0.0815,0.351,162.93500000000000,audio_features,7ssFG94tZHpX9vbIFM0GVH,spotify:track:7ssFG94tZHpX9vbIFM0GVH,https://api.spotify.com/v1/tracks/7ssFG94tZHpX...,https://api.spotify.com/v1/audio-analysis/7ssF...,201720,4,No Excuses
3,0.481,0.852,11,-4.109,0,0.4410,0.6610,0.00000000,0.3650,0.537,84.79900000000001,audio_features,1SDtZ69uALD5zW8k9S8hTf,spotify:track:1SDtZ69uALD5zW8k9S8hTf,https://api.spotify.com/v1/tracks/1SDtZ69uALD5...,https://api.spotify.com/v1/audio-analysis/1SDt...,302120,4,Nate
4,0.812,0.784,2,-4.208,1,0.2480,0.1360,0.00000000,0.1090,0.380,145.99500000000000,audio_features,18czZN7uruOjftj71Kt8oj,spotify:track:18czZN7uruOjftj71Kt8oj,https://api.spotify.com/v1/tracks/18czZN7uruOj...,https://api.spotify.com/v1/audio-analysis/18cz...,211644,4,PAID MY DUES
5,0.653,0.828,5,-4.502,0,0.3120,0.4060,0.00000000,0.1100,0.709,75.68400000000000,audio_features,2uEuo87JC9tc1dzmPUtOxj,spotify:track:2uEuo87JC9tc1dzmPUtOxj,https://api.spotify.com/v1/tracks/2uEuo87JC9tc...,https://api.spotify.com/v1/audio-analysis/2uEu...,232053,4,Returns
6,0.585,0.551,5,-10.614,0,0.0294,0.4240,0.90900000,0.0884,0.078,146.00399999999999,audio_features,4Oao30MywlksuDENDH40wS,spotify:track:4Oao30MywlksuDENDH40wS,https://api.spotify.com/v1/tracks/4Oao30Mywlks...,https://api.spotify.com/v1/audio-analysis/4Oao...,299178,4,The Dark
7,0.789,0.789,2,-4.813,1,0.2980,0.5980,0.00000000,0.0990,0.370,119.95000000000000,audio_features,3oLe5ZILASG8vU5dxIMfLY,spotify:track:3oLe5ZILASG8vU5dxIMfLY,https://api.spotify.com/v1/tracks/3oLe5ZILASG8...,https://api.spotify.com/v1/audio-analysis/3oLe...,248053,4,The Search
8,0.501,0.949,2,-11.106,1,0.0864,0.0822,0.93500000,0.1290,0.106,110.15500000000000,audio_features,5RnePZ7MW2LHyJLB7H9GpT,spotify:track:5RnePZ7MW2LHyJLB7H9GpT,https://api.spotify.com/v1/tracks/5RnePZ7MW2LH...,https://api.spotify.com/v1/audio-analysis/5Rne...,321882,5,The Visitor
9,0.868,0.654,9,-4.832,0,0.0766,0.2080,0.00000000,0.1870,0.299,138.02000000000001,audio_features,5lehoWkVPujeOAwb8BO0uK,spotify:track:5lehoWkVPujeOAwb8BO0uK,https://api.spotify.com/v1/tracks/5lehoWkVPuje...,https://api.spotify.com/v1/audio-analysis/5leh...,225440,4,Only


In [278]:
list(user_df.loc[user_df['track_name'] == 'Nate', 'uri'])

['spotify:track:1SDtZ69uALD5zW8k9S8hTf']

In [238]:
tracks = sp.current_user_top_tracks()
track_names = [track['name'] for track in tracks['items']]
track_names

['When I Grow Up',
 'Blockbuster Night, Pt. 1',
 'No Excuses',
 'Nate',
 'PAID MY DUES',
 'Returns',
 'The Dark',
 'The Search',
 'The Visitor',
 'Only',
 'Leave Me Alone',
 'Faithless',
 'FYM (feat. Boosie BadAzz)',
 'Options',
 'Looking Down the Barrel of Today',
 'Order out of Chaos',
 'Let Me Go',
 'Venom - Music From The Motion Picture',
 'WHY',
 'La La Land (feat. YG)']

In [212]:
from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity
pd.set_option("display.precision", 14)

In [310]:
def compare_songs(df, user_df):
    
    band_track_names = df['track_name']
    user_track_names = user_df['track_name']

    df_numeric = df.loc[:, df.columns[:11]]
    user_df_numeric = user_df.loc[:, user_df.columns[:11]]
    
    recs = pairwise_distances(df_numeric, user_df_numeric, metric='cosine')
    rec_df = pd.DataFrame(recs, columns=user_track_names, index=band_track_names)    
    rec_df = 1 - rec_df
    rec_df.drop_duplicates(inplace=True)
    
    song_list = []
    
    for user_track in rec_df.columns:
        max_cosine = rec_df[user_track].max()
        song_name = rec_df[rec_df[user_track] == max_cosine].index[0]
        song_uri = list(df.loc[df['track_name'] == song_name, 'uri'])[0]
        print(f'{user_track}------ closest to ---------> {song_name}')
        song_list.append((song_name, song_uri))
    
    return song_list, rec_df

In [311]:
compare_songs(df, user_df)

KeyError: 'track_name'

# Test out Functions!
get_artist_tracks
get_user_tracks
compare_songs

In [307]:
artist_tracks = get_artist_tracks(sp, artists = ['spotify:artist:0U04xAocTetRnA7v3TYQ2i', run_the_jewels])
user_tracks = get_user_tracks(sp)

In [312]:
best_tracks, df = compare_songs(artist_tracks, user_tracks)

When I Grow Up------ closest to ---------> Shotgun (Single Version)
Blockbuster Night, Pt. 1------ closest to ---------> Blockbuster Night, Pt. 1
No Excuses------ closest to ---------> Get It
Nate------ closest to ---------> The Blues
PAID MY DUES------ closest to ---------> Stay Hungry
Returns------ closest to ---------> Hey Kids
The Dark------ closest to ---------> Traveller
The Search------ closest to ---------> Pew Pew Pew - Bonus Track
The Visitor------ closest to ---------> Chic Cheer
Only------ closest to ---------> ooh la la (feat. Greg Nice & DJ Premier)


In [314]:
best_tracks

[('Shotgun (Single Version)', 'spotify:track:0WjU4Zw1aBu0hhY1rF4Avo'),
 ('Blockbuster Night, Pt. 1', 'spotify:track:6zqYITGsQw58qXz31AtQah'),
 ('Get It', 'spotify:track:63i0xlsKuhhHqt9fmpRrMJ'),
 ('The Blues', 'spotify:track:29DDoCgvMIumUG5sxQXs4t'),
 ('Stay Hungry', 'spotify:track:5W3wdlqAEOZbsPTDZ7Nt4v'),
 ('Hey Kids', 'spotify:track:2ZPHQZkhWNNjKQ32d8aimv'),
 ('Traveller', 'spotify:track:4iDVaWr7aeYkcZrv2qQS3f'),
 ('Pew Pew Pew - Bonus Track', 'spotify:track:1UwINkY4CL6d0WXgmgdGtv'),
 ('Chic Cheer', 'spotify:track:02kvYfalofG2leypGUvu3z'),
 ('ooh la la (feat. Greg Nice & DJ Premier)',
  'spotify:track:5taqLrLouA4vCjM7ZQpEtW')]